#### Question 6. Serving the LLM. Number of output tokens 

In [ ]:
!docker build -t ollama-gemma2b .

In [ ]:
!docker run --rm -p 11434:11434 ollama-gemma2b

In [ ]:
!rm -f minsearch.py
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

In [1]:
import requests 
import minsearch

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

index.fit(documents)

In [2]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=3
    )

    return results

In [4]:
def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [ ]:
!pip install accelerate

In [17]:
from transformers import AutoTokenizer, AutoModelForCausalLM

ModuleNotFoundError: No module named 'transformers'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b")
model = AutoModelForCausalLM.from_pretrained("google/gemma-2b", device_map="auto")

In [3]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

def llm(prompt):
    response = client.chat.completions.create(
        model='gemma:2b',
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    
    return response.choices[0].message.content



def llm(prompt):
    input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
    outputs = model.generate(input_ids, )
    result = tokenizer.decode(outputs[0])
    return result


In [5]:
from openai import OpenAI

client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key='ollama',
)

In [6]:
!docker ps

CONTAINER ID   IMAGE            COMMAND               CREATED          STATUS          PORTS                                           NAMES
170f87cbd9cd   ollama-gemma2b   "/bin/ollama serve"   48 seconds ago   Up 47 seconds   0.0.0.0:11434->11434/tcp, :::11434->11434/tcp   trusting_lewin


In [8]:
!docker exec trusting_lewin ollama pull gemma:2b

pulling manifest ⠋ pulling manifest ⠹ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


In [9]:
llm("What's the formula for energy?")

"Sure, here's the formula for energy:\n\n**E = K + U**\n\nWhere:\n\n* **E** is the energy in joules (J)\n* **K** is the kinetic energy in joules (J)\n* **U** is the potential energy in joules (J)\n\n**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:\n\n**K = 1/2mv^2**\n\n**Potential energy (U)** is the energy an object possesses due to its position or configuration. It is calculated as the product of an object's mass, gravitational constant (g), and height or position above a reference point.\n\n**U = mgh**\n\nWhere:\n\n* **m** is the mass in kilograms (kg)\n* **g** is the gravitational constant (9.8 m/s^2)\n* **h** is the height or position in meters (m)\n\nThe formula shows that energy can be expressed as the sum of kinetic and potential energy. The kinetic energy is a measure of the object's ability to do work, while the potential energy is a measure of the

In [10]:
print(_)

Sure, here's the formula for energy:

**E = K + U**

Where:

* **E** is the energy in joules (J)
* **K** is the kinetic energy in joules (J)
* **U** is the potential energy in joules (J)

**Kinetic energy (K)** is the energy an object possesses when it moves or is in motion. It is calculated as half the product of an object's mass (m) and its velocity (v) squared:

**K = 1/2mv^2**

**Potential energy (U)** is the energy an object possesses due to its position or configuration. It is calculated as the product of an object's mass, gravitational constant (g), and height or position above a reference point.

**U = mgh**

Where:

* **m** is the mass in kilograms (kg)
* **g** is the gravitational constant (9.8 m/s^2)
* **h** is the height or position in meters (m)

The formula shows that energy can be expressed as the sum of kinetic and potential energy. The kinetic energy is a measure of the object's ability to do work, while the potential energy is a measure of the object's ability to do w

In [ ]:
!pip install tiktoken

In [14]:
from tiktoken import TikToken

# Assuming 'gemma:2b' is your model's identifier and it requires specific tokenizer settings
tokenizer = TikToken.get_encoding('gemma:2b')

# Now you can use this tokenizer to encode or decode as needed
text = "Example text for tokenization."
tokens = tokenizer.encode(text)

ImportError: cannot import name 'TikToken' from 'tiktoken' (/usr/local/python/3.10.13/lib/python3.10/site-packages/tiktoken/__init__.py)

In [13]:
import tiktoken

encoding = tiktoken.encoding_for_model('gemma:2b')

# Encoding the text and counting the tokens
token = encoding.encode(prompt)
num_tokens = len(token)

print("Number of tokens in the query:", num_tokens)

decoded= encoding.decode_single_token_bytes(63842)
print("Decoded example token:", decoded)

KeyError: 'Could not automatically map gemma:2b to a tokeniser. Please use `tiktoken.get_encoding` to explicitly get the tokeniser you expect.'